This code without ML gives Public score: 4.887 and Private score: 5.933

In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
PATH = ('../input/tabular-playground-series-mar-2022/train.csv')

In [ ]:
train_data = pd.read_csv('../input/tabular-playground-series-mar-2022/train.csv', index_col='row_id')
test_data = pd.read_csv('../input/tabular-playground-series-mar-2022/test.csv', index_col='row_id')

#### Feature engineering

In [ ]:
train_data['time'] = pd.to_datetime(train_data.time)
test_data['time'] = pd.to_datetime(test_data.time)

In [ ]:
y_train = train_data.congestion
train_data.drop('congestion', axis=1, inplace=True)

In [ ]:
def get_time_feature(data):
    data['month'] = data.time.apply(lambda x: x.month)
    data['day'] = data.time.apply(lambda x: x.day)
    data['hour'] = data.time.apply(lambda x: x.hour)
    data['minutes'] = data.time.apply(lambda x: x.minute)
    data['dow'] = data.time.apply(lambda x: x.weekday())
    data['is_weekend'] = data.time.apply(lambda x: 1 if x.weekday() in [5,6] else 0)
    return data.drop('time', axis=1)

In [ ]:
X_train = get_time_feature(train_data)
X_test = get_time_feature(test_data)

In [ ]:
X_target = X_train.copy()
X_target['target'] = y_train

In [ ]:
#join coodinates and direction
X_target['joint_dir'] = X_target[['x', 'y', 'direction']].astype(str).apply(lambda x: ''.join(x.values), axis=1)

In [ ]:
#hour separation
def hour_sep(x):
    if (x>=12)&(x<15):
        return '[12:15]'
    elif (x>=15)&(x<18):
        return '[15:18]'
    elif (x>=18)&(x<21):
        return '[18:21]'
    elif (x>=21):
        return '[21:24]'
    else:
        return '[0:12]'

In [ ]:
X_target['hour_sep'] = X_target.hour.apply(lambda x: hour_sep(x))

#### Cosine distance: compare september with other months

In [ ]:
from scipy.spatial.distance import cosine

In [ ]:
# skipped why it happened to make such filter 
data = X_target[(X_target.is_weekend==0)&(X_target.dow!=4)&(X_target.hour>=12)]

Creation a table to fill with cosine distance

In [ ]:
arr1, arr2 =[], []
for item in data.joint_dir.unique():
    arr1.extend((item+' '+item+' '+item+' '+item).split(' '))
    arr2.extend(data.hour_sep.unique())

In [ ]:
arrays = [
    np.array(arr1), 
    np.array(arr2)
]

In [ ]:
# DataFrame with len 260: 65 unique values of 'joint_dir' and 4 unique values of 'hour_sep'
table = pd.DataFrame(data = np.zeros(shape=(260,5)), columns=[4, 5, 6, 7, 8], index = arrays)

In [ ]:
# fill DataFrame with cosine distances
for joint_d in data.joint_dir.unique():
    for hour_s in data.hour_sep.unique():
        for mon in table.columns:
            table.loc[(joint_d,hour_s), mon] = round(cosine(
                data[(data.joint_dir==joint_d)&(data.hour_sep==hour_s)&(data.month==mon)].groupby(['hour', 'minutes']).target.median().values,
                data[(data.joint_dir==joint_d)&(data.hour_sep==hour_s)&(data.month==9)].groupby(['hour', 'minutes']).target.median().values
            ), 5)
            

Drop data if distance exceeds mean in 2.8 times. The mean is calculated by averaging the first 3 lowest values of distances through each row.

In [ ]:
# collect indexes to drop
l=[]
for indx in range(260):
    mean_ = table.iloc[indx].sort_values()[:3].mean()
    mon = table.iloc[indx][table.iloc[indx]/mean_>=2.8].index.values
    joint_d = table.iloc[indx].name[0]
    hour_s = table.iloc[indx].name[1]
    if list(mon):
        for m in mon:
            l.extend(data[(data.joint_dir==joint_d)&(data.hour_sep==hour_s)&(data.month==m)].index.values)

In [ ]:
train_dropped = data.drop(l)

In [ ]:
pivot =pd.pivot_table(train_dropped, values='target', index = ['x', 'y', 'direction', 'hour', 'minutes'], columns='month', aggfunc='median').reset_index()
pivot['mean'] = round(pivot.iloc[:, 5:11].apply(np.mean, axis=1), 0)

#### Submission

In [ ]:
features = ['x', 'y', 'direction', 'hour', 'minutes']

In [ ]:
X_test = X_test[features].copy().sort_values(by=features)

In [ ]:
X_test['congestion'] = pivot['mean'].values
X_test.sort_index(inplace=True)
X_test.drop(columns=features, inplace=True)

In [ ]:
X_test.to_csv('congestion.txt')